<a href="https://colab.research.google.com/github/Gnomasss/tinkoffEDA/blob/main/tinkoffCSGO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


In [82]:
#base_dir = './drive/MyDrive/datasets/tinkoff/csgo/'
base_dir = './csgo/'

results_df = pd.read_csv(base_dir+'results.csv',low_memory=False)
#picks_df = pd.read_csv(base_dir+'picks.csv',low_memory=False)
#economy_df = pd.read_csv(base_dir+'economy.csv',low_memory=False)
#players_df = pd.read_csv(base_dir+'players.csv',low_memory=False)

In [83]:
results_df.head()

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
0,2020-03-18,Recon 5,TeamOne,Dust2,0,16,2,2,0,1,0,15,5151,2340454,62,63,0,2,2
1,2020-03-18,Recon 5,TeamOne,Inferno,13,16,2,2,8,6,5,10,5151,2340454,62,63,0,2,2
2,2020-03-18,New England Whalers,Station7,Inferno,12,16,2,1,9,6,3,10,5243,2340461,140,118,12,16,2
3,2020-03-18,Rugratz,Bad News Bears,Inferno,7,16,2,2,0,8,7,8,5151,2340453,61,38,0,2,2
4,2020-03-18,Rugratz,Bad News Bears,Vertigo,8,16,2,2,4,5,4,11,5151,2340453,61,38,0,2,2


In [84]:
min_rank = 30
results_df = results_df[(results_df.rank_1<min_rank)&(results_df.rank_2<min_rank)]

#picks_df     = picks_df  [picks_df  .match_id.isin(results_df.match_id.unique())]
#economy_df   = economy_df[economy_df.match_id.isin(results_df.match_id.unique())]
#players_df   = players_df[players_df.match_id.isin(results_df.match_id.unique())]

In [85]:
ct_1 = results_df[['date','_map','ct_1']].rename(columns={'ct_1':'ct'})
ct_2 = results_df[['date','_map','ct_2']].rename(columns={'ct_2':'ct'})
ct = pd.concat((ct_1,ct_2))

In [86]:
t_1 = results_df[['date','_map','t_1']].rename(columns={'t_1':'t'})
t_2 = results_df[['date','_map','t_2']].rename(columns={'t_2':'t'})
t = pd.concat((t_1,t_2))

In [87]:
t = t.sort_values('date')
ct = ct.sort_values('date')

In [88]:
t

,date,_map,t
45772,2015-11-03,Cobblestone,12
45771,2015-11-03,Train,1
45771,2015-11-03,Train,12
45772,2015-11-03,Cobblestone,6
45770,2015-11-04,Inferno,9
...,...,...,...
27,2020-03-17,Dust2,8
26,2020-03-17,Inferno,2
25,2020-03-17,Mirage,5
27,2020-03-17,Dust2,2


In [89]:
maps = ['Cache','Cobblestone','Dust2','Inferno','Mirage','Nuke','Overpass','Train','Vertigo']

In [90]:
series_t, series_ct, how_ct = {},{},{}
for i, key in enumerate(maps):
    t_map = t[t._map == maps[i]]
    ct_map = ct[ct._map == maps[i]]
    y_t = t_map.t.rolling(min_periods = 20, window= 200, center=True).sum().values
    y_ct = ct_map.ct.rolling(min_periods = 20, window= 200, center=True).sum().values
    
    series_t[key] = pd.Series(data=y_t,index=t_map.date)
    series_ct[key] = pd.Series(data=y_ct,index=ct_map.date)
    
    how_ct[key] = series_ct[key]/(series_ct[key]+series_t[key])//0.001/10
    print(how_ct[key])

date
2015-11-05    44.1
2015-11-05    44.2
2015-11-05    44.3
2015-11-05    44.4
2015-11-06    44.5
              ... 
2019-05-03    48.5
2019-05-05    48.7
2019-05-05    49.0
2019-05-09    48.9
2019-05-09    49.1
Length: 1800, dtype: float64
date
2015-11-03    46.3
2015-11-03    46.1
2015-11-05    46.1
2015-11-05    46.0
2015-11-06    45.9
              ... 
2018-04-29    46.6
2018-05-05    46.6
2018-05-05    46.7
2018-05-05    46.8
2018-05-05    46.6
Length: 1796, dtype: float64
date
2015-11-05    52.4
2015-11-05    52.7
2015-11-07    52.5
2015-11-07    52.6
2015-11-07    52.4
              ... 
2020-03-08    48.6
2020-03-17    48.6
2020-03-17    48.8
2020-03-17    48.7
2020-03-17    48.6
Length: 1784, dtype: float64
date
2015-11-04    56.1
2015-11-04    56.2
2015-11-05    56.3
2015-11-05    56.5
2015-11-07    56.4
              ... 
2020-03-15    49.0
2020-03-16    48.8
2020-03-16    48.9
2020-03-17    48.7
2020-03-17    48.8
Length: 2650, dtype: float64
date
2015-11-06    52.9
2015

In [91]:
def add_trace(_map):
    fig.add_trace(go.Scatter(x=how_ct[_map].index, y=how_ct[_map].values, name=_map))

In [92]:
fig = go.Figure()
for _map in maps:
    add_trace(_map)
fig.add_trace(go.Scatter(x=['2015-11-01', '2020-03-12'], y=[50,50],
                         mode='lines',line=dict(color='grey'),showlegend=False))
fig.update_layout(title='Distribution of rounds between CT and T sides',
                  yaxis_title='Percentage of round won on the CT-side (%)')
fig.show()

Продублирую тут то, что писал в ответе.
Nuke и Train имеют перевес в балансе в сторону counter terrorist, в то время как Dust 2 и Cache в сторону их противников. Inferno после 2016 года стала одной из самых сбалансированных карт. 